In [ ]:
import pandas as pd

#  โหลดและคลีนข้อมูลสภาพอากาศจากทุกชีท
file_path = 'rain.xlsx'
xls = pd.ExcelFile(file_path)

df_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df = df.rename(columns={'ปี': 'year', 'เดือน': 'month', 'วันที่': 'day'})  # เปลี่ยนชื่อเป็นอังกฤษ

    #  แปลงวันที่เป็นตัวเลข และกรองค่าที่ว่าง
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['month'] = pd.to_numeric(df['month'], errors='coerce')
    df['day'] = pd.to_numeric(df['day'], errors='coerce')
    df = df.dropna(subset=['year', 'month', 'day'])

    # แปลงเป็น datetime และกรองค่าที่แปลงไม่ได้
    df['วัน'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')
    df = df.dropna(subset=['วัน'])

    #  แปลงคอลัมน์อุณหภูมิ/ฝน/ชื้น เป็นตัวเลข (0 ไม่ถูกลบ)
    for col in ['อุณหภูมิ', 'ฝน', 'ชื้น']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    #  กรองเฉพาะแถวที่ไม่มี NaN ในคอลัมน์หลัก (แต่ 0 ยังอยู่)
    df = df[df[['อุณหภูมิ', 'ฝน', 'ชื้น']].notna().all(axis=1)]

    df_list.append(df)

#  รวมทุกชีทเข้าด้วยกัน
weather = pd.concat(df_list, ignore_index=True)
weather = weather.drop_duplicates()
weather = weather.sort_values('วัน').reset_index(drop=True)

#  บันทึกไฟล์ที่คลีนแล้ว
weather.to_excel('rain_all_years_sorted.xlsx', index=False)
print(" บันทึกไฟล์ rain_all_years_sorted.xlsx เรียบร้อยแล้ว")

#  โหลดและคลีนข้อมูลผู้ป่วย
xls = pd.ExcelFile('DHFcc12.xlsx')
df_list = []

for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df.columns = df.columns.str.strip()  # ลบช่องว่างจากชื่อคอลัมน์

    #  แปลงวันเริ่มป่วยเป็น datetime
    if 'วันเริ่มป่วย' in df.columns:
        df['วันเริ่มป่วย'] = pd.to_datetime(df['วันเริ่มป่วย'], errors='coerce')

    #  แปลงอายุเป็นตัวเลข (0 ได้)
    for col in ['อายุ(ปี)', 'อายุ(เดือน)']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    df_list.append(df)

#  รวมข้อมูลผู้ป่วย
patients = pd.concat(df_list, ignore_index=True)
patients = patients.drop_duplicates()
patients = patients.dropna(subset=['วันเริ่มป่วย'])

print(f" จำนวนผู้ป่วยที่มีวันเริ่มป่วย: {len(patients)}")

#  ฟังก์ชันคำนวณ rolling average
def get_rolling_avg(date, n_days=15):
    start_date = date - pd.Timedelta(days=n_days)
    mask = (weather['วัน'] >= start_date) & (weather['วัน'] < date)
    subset = weather.loc[mask]
    if subset.empty:
        return pd.Series([None, None, None])
    return pd.Series([
        subset['อุณหภูมิ'].mean(),
        subset['ฝน'].mean(),
        subset['ชื้น'].mean()  #ฟังก์ชันนี้รับวันที่ date แล้วคืนค่าเฉลี่ยอุณหภูมิ ฝน และความชื้น ย้อนหลัง n_days วัน
    ])

#  เพิ่มค่าเฉลี่ยย้อนหลัง 15 วัน
patients[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']] = patients['วันเริ่มป่วย'].apply(get_rolling_avg)

print(" คำนวณค่าเฉลี่ย temp, rain และ humid ย้อนหลัง 15 วันเสร็จสิ้น")

# สร้างคอลัมน์ ปี เดือน ไตรมาส ฤดูกาล
patients['ปี'] = patients['วันเริ่มป่วย'].dt.year
patients['เดือน'] = patients['วันเริ่มป่วย'].dt.month
patients['ไตรมาส'] = patients['วันเริ่มป่วย'].dt.quarter

#  ฟังก์ชันแบ่งฤดูแบบไทย
def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day

    if (month == 2 and day >= 15) or month in [3, 4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6, 7, 8, 9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

patients['ฤดูกาล'] = patients.apply(assign_season, axis=1)

#  สรุปรายวัน
if 'วันพบผป' in patients.columns:
    daily_counts = patients.groupby('วันพบผป').size().reset_index(name='จำนวนผู้ป่วย')

    daily_weather = patients.groupby('วันพบผป').agg({
        'temp_15d_avg': 'mean',
        'rain_15d_avg': 'mean',
        'humid_15d_avg': 'mean'
    }).reset_index()

    df_analysis = pd.merge(daily_counts, daily_weather, on='วันพบผป', how='left')

    daily_vars = patients.groupby('วันพบผป').agg({
        'อายุ(ปี)': 'mean',
        'อายุ(เดือน)': 'mean',
        'เพศ': lambda x: x.mode()[0] if not x.mode().empty else None,
        'อาชีพ': lambda x: x.mode()[0] if not x.mode().empty else None,
        'ชื่อหมู่บ้าน': lambda x: x.mode()[0] if not x.mode().empty else None,
        'ตำบล': lambda x: x.mode()[0] if not x.mode().empty else None,
        'อำเภอ': lambda x: x.mode()[0] if not x.mode().empty else None,
        'เดือน': lambda x: x.mode()[0] if not x.mode().empty else None,
        'ปี': lambda x: x.mode()[0] if not x.mode().empty else None,
        'ไตรมาส': lambda x: x.mode()[0] if not x.mode().empty else None,
        'ฤดูกาล': lambda x: x.mode()[0] if not x.mode().empty else None
    }).reset_index()

    df_analysis = pd.merge(df_analysis, daily_vars, on='วันพบผป', how='left')
    print("✅ รวมข้อมูลผู้ป่วยกับสภาพอากาศรายวันเสร็จแล้ว")
else:
    print("⚠️ ไม่พบคอลัมน์ 'วันพบผป' ในข้อมูลผู้ป่วย")


In [ ]:
print("=== ตรวจสอบจำนวนและเปอร์เซ็นต์ของค่า 0 ในทุกคอลัมน์ ===")

for col in df_analysis.columns:
    # ดึงข้อมูลที่ไม่ใช่ NaN
    series = df_analysis[col].dropna()

    # เช็คว่าค่าประเภทในคอลัมน์นั้นสามารถเทียบกับ 0 ได้หรือไม่ (เช่น ตัวเลข)
    try:
        n_zeros = (series == 0).sum()
        perc_zeros = n_zeros / len(series) * 100
        print(f"{col}: 0 จำนวน = {n_zeros}, 0 เปอร์เซ็นต์ = {perc_zeros:.2f}%")
    except Exception as e:
        print(f"{col}: ไม่สามารถตรวจสอบค่า 0 ได้ ({str(e)})")


In [ ]:
print(daily_vars.head())


# ตรวจสอบความสัมพันธ์ (Correlation / Chi-Square)

In [ ]:
df_analysis.rename(columns={'จำนวนผู้ป่วย': 'cases'}, inplace=True)


In [ ]:
df_analysis.rename(columns={
    'temp_15d_avg': 'temp',
    'rain_15d_avg': 'rain',
    'humid_15d_avg': 'humidity',
    'ชื่อหมู่บ้าน': 'village',
    'ตำบล': 'sub_district',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ปี': 'year',
    'ไตรมาส': 'quarter',
    'ฤดูกาล': 'season'
}, inplace=True)


In [ ]:
from scipy.stats import chi2_contingency

# --- แยกตัวแปร ---
numeric_vars = ['temp', 'rain', 'humidity']
categorical_vars = ['village', 'sub_district', 'district', 'month', 'year', 'quarter', 'season']

# --- สร้างคอลัมน์กลุ่มจำนวนผู้ป่วยแบบแบ่ง 3 กลุ่ม ---
df_analysis['จำนวนผู้ป่วยกลุ่ม'] = pd.qcut(df_analysis['cases'], q=3, labels=False, duplicates='drop')

# === Correlation ===
print("=== 📊 Numeric Variables Correlation with จำนวนผู้ป่วย ===")
for var in numeric_vars:
    corr = df_analysis[var].corr(df_analysis['cases'])
    print(f"{var} vs จำนวนผู้ป่วย: correlation = {corr:.4f}")

# === Chi-square test ===
print("\n=== 🧪 Categorical Variables Association with จำนวนผู้ป่วย (Chi-square test) ===")
for var in categorical_vars:
    table = pd.crosstab(df_analysis[var], df_analysis['จำนวนผู้ป่วยกลุ่ม'])
    if table.shape[0] < 2 or table.shape[1] < 2:
        print(f"{var} – ข้อมูลไม่พอสำหรับ Chi-square")
        continue
    chi2, p, dof, ex = chi2_contingency(table)
    print(f"{var} vs จำนวนผู้ป่วย (กลุ่ม): p-value = {p:.4f}")

# === เช็ค Poisson dispersion ===
mean_count = df_analysis['cases'].mean()
var_count = df_analysis['cases'].var()
dispersion_ratio = var_count / mean_count

print("\n=== 🧮 Poisson dispersion check ===")
print(f"Mean จำนวนผู้ป่วย = {mean_count:.4f}")
print(f"Variance จำนวนผู้ป่วย = {var_count:.4f}")
print(f"Dispersion ratio (variance/mean) = {dispersion_ratio:.4f}")

if dispersion_ratio > 1.5:
    print("มี overdispersion (variance > mean) แนะนำใช้ Negative Binomial model")
else:
    print("ไม่มี overdispersion โมเดล Poisson ใช้ได้")


In [ ]:
print(df_analysis.columns.tolist())


# Regression Modeling (Poisson และ NB)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# สูตรโมเดลเดิมที่ใช้
formula = 'cases ~ temp + rain + humidity + C(village) + C(sub_district) + C(district) + C(month) + C(year) + C(quarter) + C(season)'

# Fit Negative Binomial model
negbin_model = smf.glm(formula=formula, data=df_analysis, family=sm.families.NegativeBinomial()).fit()

# ดู summary ผลลัพธ์โมเดล Negative Binomial
print(negbin_model.summary())


In [ ]:
print(df_analysis.dtypes)
print(df_analysis[['season', 'temp', 'rain', 'humidity']].isna().sum())


In [ ]:
df_analysis['village'] = df_analysis['village'].astype('category')
df_analysis['district'] = df_analysis['district'].astype('category')


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

formula = 'cases ~ temp + rain + humidity + C(village) + C(sub_district) + C(district) + C(year) + C(season)'

negbin_model = smf.glm(formula=formula, data=df_analysis, family=sm.families.NegativeBinomial()).fit()

print(negbin_model.summary())


In [ ]:
print(df_analysis.dtypes)


In [ ]:
print(df_analysis['season'].cat.categories)
print(df_analysis['village'].cat.categories)
print(df_analysis['sub_district'].cat.categories)
print(df_analysis['district'].cat.categories)


In [ ]:
print(df_analysis.isna().sum())


# ดึงตัวแปรที่มีนัยสำคัญ

In [ ]:
import pandas as pd

results_df = pd.DataFrame({
    'coef': negbin_model.params,
    'std_err': negbin_model.bse,
    'z': negbin_model.tvalues,
    'p_value': negbin_model.pvalues,
    'conf_low': negbin_model.conf_int().iloc[:, 0],
    'conf_high': negbin_model.conf_int().iloc[:, 1]
})

significant_vars = results_df[results_df['p_value'] < 0.05]

print("=== ตัวแปรที่มีความสำคัญทางสถิติ (p < 0.05) ===")
print(significant_vars)


# Machine Learning ด้วย XGBoost & Random Forest

In [ ]:
pip install joblib


In [ ]:
# เลือกเฉพาะตัวแปรที่สำคัญ (ในที่นี้คือ dummy variables ของ year 2023 และ 2024)
selected_features = ['year']  # ก่อนอื่นเอา 'year' เข้าไปก่อน

# แปลง categorical 'year' เป็น dummy variable
X = pd.get_dummies(df_analysis[selected_features], drop_first=True)

# ตัวแปรเป้าหมาย
y = df_analysis['cases']

# แบ่งข้อมูล
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Scaling สำหรับ Deep Learning
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# XGBoost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost")
print(" RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_xgb)))
print(" R2  :", r2_score(y_test, y_pred_xgb))


# Random Forest
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("\nRandom Forest")
print(" RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print(" R2  :", r2_score(y_test, y_pred_rf))


# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

dl_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

dl_model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

dl_model.fit(X_train_scaled, y_train,
             validation_split=0.2,
             epochs=100,
             batch_size=16,
             verbose=0,
             callbacks=[early_stop])

y_pred_dl = dl_model.predict(X_test_scaled).flatten()

print("\nDeep Learning (ปรับปรุง)")
print(" RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_dl)))
print(" R2  :", r2_score(y_test, y_pred_dl))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ข้อมูล
models = ['XGBoost', 'Random Forest', 'Deep Learning']
rmse = [2.8410, 2.8401, 2.8416]
r2 = [0.1859, 0.1864, 0.18560]

x = np.arange(len(models))  # ตำแหน่งแกน x
width = 0.35  # ความกว้างของแท่งกราฟ

fig, ax = plt.subplots(figsize=(8, 5))

# แท่งกราฟ RMSE
bars1 = ax.bar(x - width/2, rmse, width, label='RMSE', color='skyblue')

# แท่งกราฟ R2
bars2 = ax.bar(x + width/2, r2, width, label='R2', color='orange')

# เพิ่มข้อความบนแท่งกราฟ
def autolabel(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # เลื่อนข้อความขึ้น 3 จุด
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10)

autolabel(bars1)
autolabel(bars2)

# กำหนดแกนและชื่อกราฟ
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()

# ใส่กริดช่วยอ่านค่า
ax.grid(axis='y', linestyle='--', alpha=0.7)

# เพิ่มหมายเหตุใต้กราฟ
plt.figtext(0.5, -0.05, 'RMSE: Root Mean Squared Error (ยิ่งต่ำยิ่งดี), R2: Coefficient of Determination (ยิ่งสูงยิ่งดี)', ha='center', fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
df_weather = pd.read_excel('rain_all_years_sorted.xlsx')

print("df_base columns:", df_base.columns.tolist())
print("df_weather columns:", df_weather.columns.tolist())


In [ ]:
#df_full.dropna(inplace=True)


# **เปรียบเทียบทีละตัว**


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- โหลดข้อมูล ---
df_base = pd.read_excel('DHFcc12.xlsx')
df_weather = pd.read_excel('rain_all_years_sorted.xlsx')

# แปลงวันให้เป็น datetime
df_base['วันเริ่มป่วย'] = pd.to_datetime(df_base['วันเริ่มป่วย'], errors='coerce')
df_weather['วัน'] = pd.to_datetime(df_weather['วัน'])
df_weather.rename(columns={'วัน': 'วันที่'}, inplace=True)

# นับจำนวนผู้ป่วยในแต่ละวัน
df_count = df_base.groupby('วันเริ่มป่วย').size().reset_index(name='จำนวนผู้ป่วย')

# คำนวณค่าเฉลี่ยย้อนหลัง 15 วัน
weather_agg = []
for date in df_count['วันเริ่มป่วย']:
    mask = (df_weather['วันที่'] >= date - pd.Timedelta(days=15)) & (df_weather['วันที่'] < date)
    past_weather = df_weather[mask]
    weather_agg.append({
        'วันเริ่มป่วย': date,
        'อุณหภูมิ': past_weather['อุณหภูมิ'].mean(),
        'ฝน': past_weather['ฝน'].mean(),
        'ชื้น': past_weather['ชื้น'].mean()
    })

df_weather_agg = pd.DataFrame(weather_agg)

# รวมข้อมูลผู้ป่วยกับค่าเฉลี่ยย้อนหลัง 15 วัน
df_full = pd.merge(df_count, df_weather_agg, on='วันเริ่มป่วย', how='left')

# สร้างฟีเจอร์จากวันที่
df_full['เดือน'] = df_full['วันเริ่มป่วย'].dt.month
df_full['ปี'] = df_full['วันเริ่มป่วย'].dt.year
df_full['ไตรมาส'] = df_full['วันเริ่มป่วย'].dt.quarter

# กำหนดฤดูกาลตามฤดูของไทย
def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day
    if (month == 2 and day >= 15) or month in [3, 4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6, 7, 8, 9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

df_full['ฤดูกาล'] = df_full.apply(assign_season, axis=1)

# สร้าง lag features (ต้องเรียงวันที่ก่อน)
df_full = df_full.sort_values('วันเริ่มป่วย').reset_index(drop=True)
df_full['จำนวนผู้ป่วย_lag1'] = df_full['จำนวนผู้ป่วย'].shift(1)
df_full['จำนวนผู้ป่วย_lag7'] = df_full['จำนวนผู้ป่วย'].shift(7)

# ลบแถวที่มี NaN (จาก lag)
df_full.dropna(inplace=True)

# Reset index อีกทีโดยไม่เรียงวันที่ เพื่อให้ train_test_split สุ่มแบ่งจริงๆ
df_full = df_full.reset_index(drop=True)

# กำหนดชุดฟีเจอร์สำหรับทดลองโมเดล
feature_sets = {
    "ข้อมูลเดิม + อุณหภูมิ": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + ความชื้น": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'ชื้น', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'ชื้น', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + อุณหภูมิ + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + ความชื้น + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'ชื้น', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'ชื้น', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
}

# สร้าง OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, drop='first')

results = []

for name, features in feature_sets.items():
    df = df_full[features + ['จำนวนผู้ป่วย']].copy()

    # ถ้ามีฤดูกาลในฟีเจอร์ ให้ทำ One-Hot Encoding
    if 'ฤดูกาล' in features:
        season_encoded = ohe.fit_transform(df[['ฤดูกาล']])
        season_cols = [f"ฤดูกาล_{cat}" for cat in ohe.categories_[0][1:]]
        df = pd.concat([df.drop(columns=['ฤดูกาล']), pd.DataFrame(season_encoded, columns=season_cols, index=df.index)], axis=1)
    else:
        season_cols = []

    X = df.drop(columns=['จำนวนผู้ป่วย'])
    y = df['จำนวนผู้ป่วย']

    # Scale เฉพาะคอลัมน์ที่ไม่ใช่ฤดูกาล (เพราะฤดูกาลเป็น one-hot แล้ว)
    num_cols = [col for col in X.columns if col not in season_cols]
    scaler = StandardScaler()
    X[num_cols] = scaler.fit_transform(X[num_cols])

    # แบ่ง train/test แบบสุ่ม
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # สร้างและเทรนโมเดล Random Forest
    model = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # ประเมินผล
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({'ชุดข้อมูล': name, 'RMSE': rmse, 'MAE': mae, 'R2': r2})

df_results = pd.DataFrame(results)
print(df_results.sort_values('R2', ascending=False).reset_index(drop=True))


In [ ]:
df_full = df_full.sort_values('วันเริ่มป่วย').reset_index(drop=True)
df_full['จำนวนผู้ป่วย_lag14'] = df_full['จำนวนผู้ป่วย'].shift(14)
df_full['จำนวนผู้ป่วย_lag30'] = df_full['จำนวนผู้ป่วย'].shift(30)

# ลบแถวที่มีค่า NaN จากการ shift
df_full = df_full.dropna().reset_index(drop=True)


# XG-BOOST

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor

# --- โหลดข้อมูล ---
df_base = pd.read_excel('DHFcc12.xlsx')
df_weather = pd.read_excel('rain_all_years_sorted.xlsx')

# --- แปลงวันให้เป็น datetime ---
df_base['วันเริ่มป่วย'] = pd.to_datetime(df_base['วันเริ่มป่วย'], errors='coerce')
df_weather['วัน'] = pd.to_datetime(df_weather['วัน'])
df_weather.rename(columns={'วัน': 'วันเริ่มป่วย'}, inplace=True)

# --- นับจำนวนผู้ป่วยในแต่ละวัน ---
df_count = df_base.groupby('วันเริ่มป่วย').size().reset_index(name='จำนวนผู้ป่วย')

# --- **เปลี่ยนตรงนี้: ไม่ใช้ค่าเฉลี่ยย้อนหลัง 15 วัน** ---
# รวมข้อมูลสภาพอากาศตรงวันนั้นโดยตรงเลย (merge ตามวันที่)
df_full = pd.merge(df_count, df_weather[['วันเริ่มป่วย', 'อุณหภูมิ', 'ฝน', 'ชื้น']], on='วันเริ่มป่วย', how='left')

# --- สร้างฟีเจอร์จากวันที่ ---
df_full['เดือน'] = df_full['วันเริ่มป่วย'].dt.month
df_full['ปี'] = df_full['วันเริ่มป่วย'].dt.year
df_full['ไตรมาส'] = df_full['วันเริ่มป่วย'].dt.quarter

# --- กำหนดฤดูกาล ---
def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day
    if (month == 2 and day >= 15) or month in [3, 4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6, 7, 8, 9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

df_full['ฤดูกาล'] = df_full.apply(assign_season, axis=1)

# --- สร้าง lag features ---
df_full = df_full.sort_values('วันเริ่มป่วย').reset_index(drop=True)
df_full['จำนวนผู้ป่วย_lag1'] = df_full['จำนวนผู้ป่วย'].shift(1)
df_full['จำนวนผู้ป่วย_lag7'] = df_full['จำนวนผู้ป่วย'].shift(7)

# --- ลบแถวที่มี NaN ---
df_full.dropna(inplace=True)
df_full = df_full.reset_index(drop=True)  # reset index หลัง dropna

# --- ชุดฟีเจอร์ ---
feature_sets = {
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'ชื้น', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'ชื้น', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + ความชื้น + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'ชื้น', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + อุณหภูมิ + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + ความชื้น": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'ชื้น', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + อุณหภูมิ": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
}

# --- One-Hot Encoding สำหรับฤดูกาล ---
ohe = OneHotEncoder(sparse_output=False, drop='first')

results = []

for name, features in feature_sets.items():
    df = df_full[features + ['จำนวนผู้ป่วย']].copy()

    if 'ฤดูกาล' in features:
        season_encoded = ohe.fit_transform(df[['ฤดูกาล']])
        season_cols = [f"ฤดูกาล_{cat}" for cat in ohe.categories_[0][1:]]
        df = pd.concat([df.drop(columns=['ฤดูกาล']), pd.DataFrame(season_encoded, columns=season_cols, index=df.index)], axis=1)
    else:
        season_cols = []

    X = df.drop(columns=['จำนวนผู้ป่วย'])
    y = df['จำนวนผู้ป่วย']

    scaler = StandardScaler()

    num_cols = [col for col in X.columns if col not in season_cols]
    X[num_cols] = scaler.fit_transform(X[num_cols])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = XGBRegressor(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        verbosity=0
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({'ชุดข้อมูล': name, 'RMSE': rmse, 'MAE': mae, 'R2': r2})

df_results = pd.DataFrame(results)
print(df_results.sort_values('R2', ascending=False).reset_index(drop=True))


# Deep learning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# ตั้ง random seed เพื่อความ reproducible
np.random.seed(42)
tf.random.set_seed(42)

# --- โหลดข้อมูล ---
df_base = pd.read_excel('DHFcc12.xlsx')
df_weather = pd.read_excel('rain_all_years_sorted.xlsx')

# แปลงวันที่เป็น datetime
df_base['วันเริ่มป่วย'] = pd.to_datetime(df_base['วันเริ่มป่วย'], errors='coerce')
df_weather['วัน'] = pd.to_datetime(df_weather['วัน'])

# เปลี่ยนชื่อคอลัมน์วันที่ใน df_weather ให้เหมือนกับ df_base เพื่อ merge
df_weather.rename(columns={'วัน': 'วันเริ่มป่วย'}, inplace=True)

# นับจำนวนผู้ป่วยแต่ละวัน
df_count = df_base.groupby('วันเริ่มป่วย').size().reset_index(name='จำนวนผู้ป่วย')

# รวมข้อมูลอากาศตรงกับวันเริ่มป่วยเลย (ไม่เอาค่าเฉลี่ยย้อนหลัง)
df_full = pd.merge(df_count, df_weather[['วันเริ่มป่วย', 'อุณหภูมิ', 'ฝน', 'ชื้น']], on='วันเริ่มป่วย', how='left')

# ลบแถวที่มีค่า NaN ในข้อมูลอากาศ
df_full.dropna(subset=['อุณหภูมิ', 'ฝน', 'ชื้น'], inplace=True)

# สร้างฟีเจอร์จากวันที่
df_full['เดือน'] = df_full['วันเริ่มป่วย'].dt.month
df_full['ปี'] = df_full['วันเริ่มป่วย'].dt.year
df_full['ไตรมาส'] = df_full['วันเริ่มป่วย'].dt.quarter

# กำหนดฤดูกาลไทย
def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day
    if (month == 2 and day >= 15) or month in [3, 4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6, 7, 8, 9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

df_full['ฤดูกาล'] = df_full.apply(assign_season, axis=1)

# สร้าง lag features
df_full = df_full.sort_values('วันเริ่มป่วย').reset_index(drop=True)
df_full['จำนวนผู้ป่วย_lag1'] = df_full['จำนวนผู้ป่วย'].shift(1)
df_full['จำนวนผู้ป่วย_lag7'] = df_full['จำนวนผู้ป่วย'].shift(7)

# ลบแถวที่มีค่า NaN จาก lag features
df_full.dropna(inplace=True)
df_full = df_full.reset_index(drop=True)  # reset index หลัง dropna

# กำหนดชุดฟีเจอร์ (เพิ่ม 'เดือน' ด้วย)
feature_sets = {
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'ชื้น', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'ชื้น', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + ความชื้น + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'ชื้น', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + อุณหภูมิ + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + ความชื้น": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'ชื้น', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
    "ข้อมูลเดิม + อุณหภูมิ": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'เดือน', 'อุณหภูมิ', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'],
}

results = []

ohe = OneHotEncoder(sparse_output=False, drop='first')

for name, features in feature_sets.items():
    df = df_full[features + ['จำนวนผู้ป่วย']].copy()

    # One-hot encoding ฤดูกาล ถ้ามีใน features
    if 'ฤดูกาล' in features:
        season_encoded = ohe.fit_transform(df[['ฤดูกาล']])
        season_cols = [f"ฤดูกาล_{cat}" for cat in ohe.categories_[0][1:]]
        df = pd.concat([df.drop(columns=['ฤดูกาล']), pd.DataFrame(season_encoded, columns=season_cols, index=df.index)], axis=1)
    else:
        season_cols = []

    X = df.drop(columns=['จำนวนผู้ป่วย'])
    y = df['จำนวนผู้ป่วย'].values

    # Scale เฉพาะฟีเจอร์ตัวเลข (ไม่รวม one-hot)
    scaler = StandardScaler()
    num_cols = [col for col in X.columns if col not in season_cols]
    X[num_cols] = scaler.fit_transform(X[num_cols])

    X = X.values

    # แบ่งข้อมูล train-test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # สร้างโมเดล Deep Learning
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0,
              validation_split=0.2, callbacks=[early_stop])

    y_pred = model.predict(X_test).flatten()

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({'ชุดข้อมูล': name, 'RMSE': rmse, 'MAE': mae, 'R2': r2})

df_results = pd.DataFrame(results)
df_results = df_results.sort_values('R2', ascending=False).reset_index(drop=True)
print(df_results)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

# โหลดข้อมูล
df_base = pd.read_excel('DHFcc12.xlsx')
df_weather = pd.read_excel('rain_all_years_sorted.xlsx')

# เตรียมวันที่
df_base['วันเริ่มป่วย'] = pd.to_datetime(df_base['วันเริ่มป่วย'], errors='coerce')
df_weather['วัน'] = pd.to_datetime(df_weather['วัน'])
df_weather.rename(columns={'วัน': 'วันเริ่มป่วย'}, inplace=True)

# รวมข้อมูล
df_count = df_base.groupby('วันเริ่มป่วย').size().reset_index(name='จำนวนผู้ป่วย')
df_full = pd.merge(df_count, df_weather[['วันเริ่มป่วย', 'อุณหภูมิ', 'ฝน', 'ชื้น']], on='วันเริ่มป่วย', how='left')
df_full.dropna(subset=['อุณหภูมิ', 'ฝน', 'ชื้น'], inplace=True)

# ฟีเจอร์จากวันที่
df_full['เดือน'] = df_full['วันเริ่มป่วย'].dt.month
df_full['ปี'] = df_full['วันเริ่มป่วย'].dt.year
df_full['ไตรมาส'] = df_full['วันเริ่มป่วย'].dt.quarter
df_full['ฤดูกาล'] = df_full['เดือน'].apply(lambda m: 'Summer' if m in [3,4,5] else 'Rainy' if m in [6,7,8,9] else 'Winter')

# lag features
df_full = df_full.sort_values('วันเริ่มป่วย').reset_index(drop=True)
df_full['จำนวนผู้ป่วย_lag1'] = df_full['จำนวนผู้ป่วย'].shift(1)
df_full['จำนวนผู้ป่วย_lag7'] = df_full['จำนวนผู้ป่วย'].shift(7)
df_full.dropna(inplace=True)

# ✅ ใช้เฉพาะฟีเจอร์นี้
features = ['ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ', 'ฝน', 'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7']
df = df_full[features + ['จำนวนผู้ป่วย']].copy()

# One-hot encoding ฤดูกาล
ohe = OneHotEncoder(sparse_output=False, drop='first')
season_encoded = ohe.fit_transform(df[['ฤดูกาล']])
season_cols = [f"ฤดูกาล_{cat}" for cat in ohe.categories_[0][1:]]
df = pd.concat([df.drop(columns=['ฤดูกาล']), pd.DataFrame(season_encoded, columns=season_cols, index=df.index)], axis=1)

# แยก X y
X = df.drop(columns=['จำนวนผู้ป่วย'])
y = df['จำนวนผู้ป่วย'].values

# Normalize
scaler = StandardScaler()
num_cols = [col for col in X.columns if col not in season_cols]
X[num_cols] = scaler.fit_transform(X[num_cols])
X = X.values

# แบ่งข้อมูล
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# เทรน XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=4, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# ประเมินผล
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# แสดงผล
print("📊 ชุดข้อมูล: ข้อมูลเดิม + อุณหภูมิ + ฝน")
print(f"RMSE: {rmse:.4f}")
print(f"MAE : {mae:.4f}")
print(f"R2  : {r2:.4f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# --- โหลดข้อมูลสภาพอากาศ ---
weather = pd.read_excel('rain_all_years_sorted.xlsx')
weather.rename(columns={'วัน': 'วันเริ่มป่วย'}, inplace=True)

# --- โหลดข้อมูลผู้ป่วย ---
df_base = pd.read_excel('DHFcc12.xlsx')
df_base['วันเริ่มป่วย'] = pd.to_datetime(df_base['วันเริ่มป่วย'], errors='coerce')
df_base = df_base.dropna(subset=['วันเริ่มป่วย'])

# --- นับจำนวนผู้ป่วยรายวัน ---
df_count = df_base.groupby('วันเริ่มป่วย').size().reset_index(name='จำนวนผู้ป่วย')

# --- ฟังก์ชัน rolling average ---
def get_rolling_avg(date, n_days=15):
    start_date = date - pd.Timedelta(days=n_days)
    mask = (weather['วันเริ่มป่วย'] >= start_date) & (weather['วันเริ่มป่วย'] < date)
    subset = weather.loc[mask]
    if subset.empty:
        return pd.Series([np.nan, np.nan, np.nan])
    return pd.Series([
        subset['อุณหภูมิ'].mean(),
        subset['ฝน'].mean(),
        subset['ชื้น'].mean()
    ])

df_count[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']] = df_count['วันเริ่มป่วย'].apply(get_rolling_avg)

# --- ดึงข้อมูลตรงวัน ---
df_count = pd.merge(df_count, weather[['วันเริ่มป่วย', 'อุณหภูมิ', 'ฝน', 'ชื้น']], on='วันเริ่มป่วย', how='left')

# --- เพิ่มคอลัมน์เวลา ---
df_count['ปี'] = df_count['วันเริ่มป่วย'].dt.year
df_count['เดือน'] = df_count['วันเริ่มป่วย'].dt.month
df_count['ไตรมาส'] = df_count['วันเริ่มป่วย'].dt.quarter

def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day
    if (month == 2 and day >= 15) or month in [3, 4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6, 7, 8, 9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

df_count['ฤดูกาล'] = df_count.apply(assign_season, axis=1)

# --- เพิ่ม lag features ---
df_count = df_count.sort_values('วันเริ่มป่วย').reset_index(drop=True)
df_count['จำนวนผู้ป่วย_lag1'] = df_count['จำนวนผู้ป่วย'].shift(1)
df_count['จำนวนผู้ป่วย_lag7'] = df_count['จำนวนผู้ป่วย'].shift(7)

# --- ลบ NaN ---
df_count = df_count.dropna().reset_index(drop=True)

# --- ฟังก์ชัน train/test และประเมินผล ---
def train_evaluate_model(feature_cols, df, model_name):
    df_model = df[feature_cols + ['จำนวนผู้ป่วย']].copy()

    if 'ฤดูกาล' in feature_cols:
        ohe = OneHotEncoder(sparse_output=False, drop='first')
        season_encoded = ohe.fit_transform(df_model[['ฤดูกาล']])
        season_cols = [f"ฤดูกาล_{cat}" for cat in ohe.categories_[0][1:]]
        df_model = pd.concat([df_model.drop(columns=['ฤดูกาล']), pd.DataFrame(season_encoded, columns=season_cols, index=df_model.index)], axis=1)
    else:
        season_cols = []

    X = df_model.drop(columns=['จำนวนผู้ป่วย'])
    y = df_model['จำนวนผู้ป่วย']

    scaler = StandardScaler()
    num_cols = [col for col in X.columns if col not in season_cols]
    X[num_cols] = scaler.fit_transform(X[num_cols])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(
        n_estimators=200,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return {
        'Model': model_name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'y_test': y_test.reset_index(drop=True),
        'y_pred': pd.Series(y_pred)
    }

# --- ฟีเจอร์ทั้งสองแบบ ---
features_rolling = [
    'ปี', 'เดือน', 'ไตรมาส', 'ฤดูกาล',
    'temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg',
    'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'
]

features_direct = [
    'ปี', 'เดือน', 'ไตรมาส', 'ฤดูกาล',
    'อุณหภูมิ', 'ฝน', 'ชื้น',
    'จำนวนผู้ป่วย_lag1', 'จำนวนผู้ป่วย_lag7'
]

# --- รันทั้งสองโมเดล ---
results = []
results.append(train_evaluate_model(features_rolling, df_count, 'Rolling 15-day average features'))
results.append(train_evaluate_model(features_direct, df_count, 'Direct daily features'))

# --- สรุปผล ---
df_results = pd.DataFrame([{
    'Model': r['Model'],
    'RMSE': r['RMSE'],
    'MAE': r['MAE'],
    'R2': r['R2']
} for r in results])
print("=== ผลการเปรียบเทียบโมเดล ===")
print(df_results)

# --- วาดกราฟเปรียบเทียบค่าจริง vs ค่าทำนาย ---
for r in results:
    plt.figure(figsize=(6, 4))
    plt.scatter(r['y_test'], r['y_pred'], alpha=0.5)
    plt.plot([r['y_test'].min(), r['y_test'].max()], [r['y_test'].min(), r['y_test'].max()], 'r--')
    plt.xlabel("ค่าจริง (Actual)")
    plt.ylabel("ค่าทำนาย (Predicted)")
    plt.title(f"{r['Model']} - ค่าจริง vs ค่าทำนาย")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
